<a href="https://colab.research.google.com/github/SRIHARITHUNGA/machine-learning/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Scraping Head & Shoulder Reviews from Amazon website.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [ ]:
def HeadShoulderReviews(url, no_of_pages):
  
  ratings = []
  review_data = []
   
  assert (url.startswith('https://www.amazon.in/')), "Only Amazon pages can be scraped using this."
  assert (url.endswith('reviewerType=all_reviews')), "Not the right page. Please use the all reviews page as the URL."

  for i in tqdm_notebook(range(1, no_of_pages + 1), desc = 'Scraping : '):
    URL = f"{url}&pageNumber={i}"
    page = requests.get(URL)

    # if page not found 404 error
    assert (page.status_code != 404), "Error 404 : Page Not Found."

     # keep requesting until successful
    while page.status_code != 200:
      page = requests.get(URL)
      
    scrape = BeautifulSoup(page.content, "html.parser")

      # get all the review cards in a pages
    cards = scrape.find_all('div', class_='a-section review aok-relative')
    # if no cards present in the page, end scraping
    if len(cards) == 0:
      print('Reached end of reviews.')
      break

    # scraping through review cards
    for card in cards:
      # try except used to add None values if value for one product review doesnt exist
      

      try:
        ratings.append(int(card.find('span', class_="a-icon-alt").text[0]))
      except:
        ratings.append(None)
      

      try:
        review_data.append(card.find('span', attrs={'class':"a-size-base review-text review-text-content", 'data-hook':'review-body'}).findChild('span').text)
      except:
        review_data.append(None)



  # converting into dictionary
  reviews = {
      'Ratings' : ratings,
      'Review' : review_data,
  }

 # creating a dataframe
  df = pd.DataFrame(reviews)

  return df

In [ ]:
url = "https://www.amazon.in/Head-Shoulders-Smooth-Silky-Shampoo/product-reviews/B00JS3PICK/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"


In [ ]:
reviews_df = HeadShoulderReviews(url, 245)
reviews_df

Scraping :   0%|          | 0/245 [00:00<?, ?it/s]

,Ratings,Review
0,4,"Product is good, silky smooth hairIssue with p..."
1,4,None
2,4,Maine ise 328 me liya great indian sale me
3,4,None
4,4,good product
...,...,...
2444,5,I give the product 5 star because this product...
2445,1,I think it is duplicate...
2446,1,"I have bought second time this product, fist t..."
2447,5,Thanks


In [ ]:
# saving to a csv file
reviews_df.to_csv('H_D_Reviews.csv', index=False)